In [1]:
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import csv
from random import randint
import sys
import re

In [2]:
headers = {"User-Agent":"Ubuntu Chromium/76.0.3809.100 Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36"}
base_url = 'https://www.anthropologie.com'

r = requests.get(base_url + '/brands', headers=headers)
soup = BeautifulSoup(r.text, 'lxml')

In [3]:
# find each brand section

brand_ids = soup.find_all('a', {'class': 'c-brands__a'}, href=True)

brand_link = []
for bi in brand_ids:
    brand_link.append(bi['href'])
    

['/brands/nude', '/brands/001-skincare-london', '/brands/1canoe2', '/brands/3x1', '/brands/52-conversations-by-anthropologie', '/brands/5x-by-ajit-kumar', '/brands/7-for-all-mankind', '/brands/8greens', '/brands/9seed', '/brands/aa-bridal', '/brands/abeego', '/brands/able-cosmetics', '/brands/ada', '/brands/adelyn-rae', '/brands/adidas-by-stella-mccartney', '/brands/adrianna-papell', '/brands/aether-beauty', '/brands/ag-hair', '/brands/ag-jeans', '/brands/agent-nateur', '/brands/agolde', '/brands/agua-bendita', '/brands/aidan-mattox', '/brands/aila-blue', '/brands/aimee-wilder', '/brands/akemi-kin', '/brands/albeit', '/brands/albetta', '/brands/aldomartins', '/brands/alex-sickling', '/brands/alexandre-de-paris', '/brands/ali-jay', '/brands/alice-mary-lynch', '/brands/alice-scott', '/brands/alicia-galer', '/brands/all-across-africa', '/brands/all-black', '/brands/all-roads-design', '/brands/allies-of-skin', '/brands/allihop', '/brands/amadi', '/brands/amadi-carpets', '/brands/amaio', '/

In [23]:
# for each brand on the anthropologie website
# find every item's url
brand_items = []
for bl in brand_link[0:9]:
    brand_url = base_url + bl
    
    r = requests.get(brand_url, headers=headers)
    soup = BeautifulSoup(r.text, 'lxml')
    
    product_list = soup.find_all('div', {'class':"c-product-tile-details c-product-tile-details--regular"})
    
    product_link = []
    for pl in product_list:
        product_href = pl.find_all('a', href=True)
        for ph in product_href:
            product_link.append(ph['href'].split('?')[0])
            
    brand_items.append(product_link)
            
    # they want me to play very very nice
    time.sleep(5)

In [39]:
all_items = [val for sublist in brand_items for val in sublist]
all_items = list(set(all_items))
all_items[0]

'/shop/3x1-high-rise-channel-seam-skinny-jeans'

In [38]:
url = base_url + all_items[0]
r = requests.get(url, headers=headers)
item_soup = BeautifulSoup(r.text, 'lxml')
print(item_soup.prettify())

<!DOCTYPE html>
<!--[if lte IE 9]><html class="u-lt-ie-10" lang="en-US"><![endif]-->
<!--[if gt IE 9]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <!-- // -->
  <script type="text/javascript">
   <!--
	document.write('<script src="/__ssobj/ard.png?6737512362984613319_1-439-'+(27339*57379+14859)+'&n=1'+'">\x3C/script\x3E');
//-->
<!-- // -->
  </script>
  <script type="text/javascript">
   window.lazySizesConfig = {customMedia: {'extra-small': '(max-width: 567px)','small': '(min-width: 568px)','medium': '(min-width: 768px)','large': '(min-width: 1025px)','extra-large': '(min-width: 1441px)'},loadMode: 0};
  </script>
  <script src="/static/lib/lazysizes/3.0.0/lazysizes-umd.min.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   var DY=DY||{};
    DY.recommendationContext = {"type": "PRODUCT", "data": ["AN-4122058740075-000"]};
  </script>
  <script src="//cdn.dynamicyield.com/api/8767825/api_dynamic.js" type="text/javascript">
  </script>
  <script src=

In [ ]:
# for each family get all products
all_products = []
for fl in all_families:
    url = ikea_base + fl
    r = requests.get(url, headers = headers)
    family_soup = BeautifulSoup(r.text, 'lxml')
    
    item_list = family_soup.find_all("div",{"class":"productLists"})
    
    item_links = []  # links to each product
    for req in item_list:
        item_hrefs = req.find_all("a", {"class":"productLink"}, href=True)
        for link_element in item_hrefs:
            item_links.append(link_element['href'])
    
    all_products.append(item_links)

    
# family_item_links is a list of lists
# list elements are each a list of product links
# flatten this list
all_products = [val for sublist in all_products for val in sublist]

In [ ]:
info_picture = []
info_corpus = []
for pp in all_products:
    url = ikea_base + pp
    r = requests.get(url, headers = headers)
    product_soup = BeautifulSoup(r.text, 'lxml')
    
    # image of product
    product_img = product_soup.find("div", {"class":"rightContentContainer"}).img
    info_picture.append(product_img['src'])
    
    # text description information
    product_info = product_soup.find_all('div', {"id":"custBenefit"}, {"class":"texts keyFeaturesmargin"})

    info_list = []
    for req in product_info:
        list_item = req.find_all('div')
        for li in list_item:
            info_list.append(li.get_text())
    
    info_list = [i.replace('-', '') for i in info_list]
    info_corpus.append(' '.join(info_list))

In [ ]:
output = pd.DataFrame(list(zip(info_picture, info_corpus)),
                      columns = ['img_url', 'description'])

output.to_csv('/home/peter/Documents/projects/insight/copyprisim/results/example.csv')